<h1>Inhaltsverzeichnis<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#D.-Systematische-Handelsstrategien" data-toc-modified-id="D.-Systematische-Handelsstrategien-1">D. Systematische Handelsstrategien</a></span></li><li><span><a href="#D.1-Ein-Asset-Strategien" data-toc-modified-id="D.1-Ein-Asset-Strategien-2">D.1 Ein-Asset-Strategien</a></span></li><li><span><a href="#D.1.2-Momentum-Strategien" data-toc-modified-id="D.1.2-Momentum-Strategien-3">D.1.2 Momentum-Strategien</a></span><ul class="toc-item"><li><span><a href="#4.-Moving-Average-Crossover:-Optimierung" data-toc-modified-id="4.-Moving-Average-Crossover:-Optimierung-3.1">4. Moving Average Crossover: Optimierung</a></span><ul class="toc-item"><li><span><a href="#Auswahl-der-Fensterlänge-für-die-gleitenden-Durchschnitte" data-toc-modified-id="Auswahl-der-Fensterlänge-für-die-gleitenden-Durchschnitte-3.1.1">Auswahl der Fensterlänge für die gleitenden Durchschnitte</a></span><ul class="toc-item"><li><span><a href="#WARNUNG:-Overfitting" data-toc-modified-id="WARNUNG:-Overfitting-3.1.1.1">WARNUNG: Overfitting</a></span></li></ul></li></ul></li></ul></li></ul></div>

# D. Systematische Handelsstrategien
# D.1 Ein-Asset-Strategien
# D.1.2 Momentum-Strategien

## 4. Moving Average Crossover: Optimierung
In Abschnitt D.1.2.2 haben wir einen einfachen Strategie-Backtest durchgeführt, in dem die Anzahl der Tage für den kurzen gleitenden Durchschnitt und den langen gleitenden Durchschnitt auf 42 bzw. 252 Tage eingestellt war. Für einen ersten Schritt, um unseren Code zu testen und sicherzustellen, dass er korrekt läuft, ist das in Ordnung, aber wie hoch ist die Wahrscheinlichkeit, dass diese beiden, relativ zufällig gewählten Periodenlängen von allen möglichen (und vernünftigen) Variationen der Perioden des gleitenden Durchschnitts die höchsten Renditen oder die höchste Sharpe-Ratio erzielen?

Nun, die einzige Möglichkeit, diese Frage zu beantworten, besteht darin mehrere Backtests mit jeweils unterschiedlichen gleitenden Durchschnittsperioden durchzuführen und die Ergebnisse zu speichern. Numpy-Arrays eignen sich hervorragend für diesen Zweck; wir können ein mehrdimensionales Array initiieren, bei dem alle Startwerte auf Null gesetzt werden, und dann, während wir durch unsere Backtests iterieren, können wir zunächst die Perioden des gleitenden Durchschnitts mit jedem Durchlauf ändern, und zusätzlich die Ergebnisse in der entsprechenden Numpy-Array-Zelle für eine spätere Analyse speichern.

Lassen Sie uns den Code mit ein paar Funktionen umstrukturieren, die wir dann später aufrufen können:

In [ ]:
#import relevant modules
import pandas as pd
import numpy as np
import pandas_datareader.data as web
from math import sqrt
import matplotlib.pyplot as plt

In [ ]:
#download data into DataFrame and create moving averages columns
sp500 = web.get_data_yahoo('^GSPC',start='1/1/2000',end='8/31/2019')

In [ ]:
#helper functions
def ma_strat(short_ma,long_ma):
    sp500['short_ma'] = np.round(sp500['Close'].rolling(window=short_ma).mean(),2)
    sp500['long_ma'] = np.round(sp500['Close'].rolling(window=long_ma).mean(),2)
     
    #create column with moving average spread differential
    sp500['short_ma-long_ma'] = sp500['short_ma'] - sp500['long_ma']
     
    #set desired number of points as threshold for spread difference and create column containing strategy 'Stance'
    X = 50
    sp500['Stance'] = np.where(sp500['short_ma-long_ma'] > X, 1, 0)
    sp500['Stance'] = np.where(sp500['short_ma-long_ma'] < -X, -1, sp500['Stance'])
    sp500['Stance'].value_counts()
     
    #create columns containing daily market log returns and strategy daily log returns
    sp500['Market Returns'] = np.log(sp500['Close'] / sp500['Close'].shift(1))
    sp500['Strategy'] = sp500['Market Returns'] * sp500['Stance'].shift(1)
     
    #set strategy starting equity to 1 (i.e. 100%) and generate equity curve
    sp500['Strategy Equity'] = sp500['Strategy'].cumsum() + 1
     
    sharpe = annualised_sharpe(sp500['Strategy'])
    
    return (sp500['Strategy'].cumsum()[-1], sharpe)

#function to calculate Sharpe Ratio - Risk free rate element excluded for simplicity
def annualised_sharpe(returns, N=252):
    return np.sqrt(N) * (returns.mean() / returns.std())


Nun können wir die Funktion "numpy.linspace()" verwenden, um ein Array zu erstellen, dem wir die verschiedenen Werte des kurzen gleitenden Durchschnittsfensters und des langen gleitenden Durchschnittsfensters, über die wir die Tests laufen lassen möchten, wie folgt zuweisen:

In [ ]:
short_ma = np.linspace(10,60,25,dtype=int)
short_ma

Da die Fensterlänge des gleitenden Durchschnitts eine ganze Zahl sein muss, fügen wir die Angabe "dtype=int" ein, die die Werte als ganze Zahlen darstellt. Was "linspace" bewirkt ist die Erzeugung eines Arrays, beginnend bei der ersten an die Funktion übergebenen Zahl und endend beim zweiten Wert, wobei die Anzahl der Zwischenschritte durch den dritten an die Funktion übergebenen Wert festgelegt wird - das obige Beispiel würde also ein Array von 25 ganzzahligen Werten mit identischem Abstand erzeugen, das bei 10 beginnt und bei 60 endet.

Wir tun dies auch für das lange Zeitfenster des gleitenden Durchschnitts:

In [ ]:
long_ma = np.linspace(220,270,25,dtype=int)
long_ma

Hier erzeugen wir ein Array aus 25 ganzen Zahlen mit gleichem Abstand, das bei 220 beginnt und bei 270 endet.

Jetzt müssen wir noch 2 Numpy-Arrays initialisieren, die die Ergebnisse unserer verschiedenen Backtest-Iterationen enthalten; ein Array für den Gewinn/Verlust der Strategie und ein Array für die Sharpe-Ratio. Beide Arrays sind zweidimensional, mit Größen/Längen, die der jeweiligen Länge unserer beiden gleitenden Durchschnittswerte-Arrays entsprechen, über die wir iterieren möchten.

In [ ]:
results_pnl = np.zeros((len(short_ma),len(long_ma)))
results_sharpe = np.zeros((len(short_ma),len(long_ma)))

Kommen wir nun zum Kern des Codes, der es uns ermöglicht, alle verschiedenen Kombinationen von kurzen und langen gleitenden Durchschnittsfensterlängen durchlaufen zu lassen:

In [ ]:
for i, shortma in enumerate(short_ma):
    for j, longma in enumerate(long_ma):
        pnl, sharpe = ma_strat(shortma,longma)
        results_pnl[i,j] = pnl
        results_sharpe[i,j] = sharpe

Bitte beachten: Dieser Code benötigt ein wenig Zeit für die Ausführung. Wenn Sie also den Code durchlaufen lassen, geben Sie ihm 5 bis 10 Minuten Zeit - der Code ist nicht sonderlich effizient und die Permutationen möglicher Kombinationen gleitender Durchschnitte sind recht groß, so dass der Code einige Male durchlaufen muss, um alle Ergebnisse zu berechnen.

Sobald der Code fertig ausgeführt ist, haben wir nun zwei Numpy Arrays "results_pnl" und "results_sharpe", die die entsprechenden Gewinn/Verlust-Daten und die Sharpe-Ratio für jeden Durchlauf des Backtests (mit entsprechenden Kombinationen aus langen und kurzen gleitenden Durchschnittsfenstern) enthalten.

Arrays sind in ihrem Rohformat etwas schwer zu analysieren - lassen Sie uns daher die Ergebnisse mit einem Farbplot visualisieren, der es uns ermöglichen sollte, zu erkennen, welche Kombinationen der Fensterlängen die besten Ergebnisse liefern. Zunächst die Ergebnisse für den Gewinn/Verlust-Array:

In [ ]:
#Colorplot for P&L
plt.pcolor(short_ma,long_ma,results_pnl.T)
plt.colorbar()
plt.show()

Wir können erkennen, dass es einen "Sweet Spot" etwas unterhalb von der Mitte zu geben scheint, was bedeutet, dass die Verwendung eines kurzen gleitenden Durchschnitts von etwa 37 Tagen und eines langen gleitenden Durchschnitts von etwa 240 Tagen die besten Ergebnisse liefert. Daneben deutet der tiefblaue Bereich auf der linken Seite an, dass die Verwendung einer Kombination von 18 Tagen und 250-260 Tagen eher nicht ratsam ist.

Wenn wir die entsprechenden Sharpe-Ratios grafisch darstellen, sehen wir fast identische Ergebnisse. Die höchste Sharpe-Ratio liefert die Kombination von 37 und 240 Tagen. Das Sharpe-Ratio Farbdiagramm wird hier dargestellt:

In [ ]:
#Colorplot for Sharpe ratio
plt.pcolor(short_ma,long_ma,results_sharpe.T)
plt.colorbar()
plt.show()

Im obigen Beispiel hatten wir nur zwei Parameter zu optimieren, und trotzdem brauchte der Code ein wenig Zeit, um ausgeführt zu werden. Sie können also sehen, dass mit zunehmender Anzahl der zu optimierenden Parameter die für die Ausführung der Iterationen benötigte Zeit exponentiell zunimmt. Mit anderen Worten, das Optimieren von mehr als ein paar Parametern durch "Brute-Force"-Methoden wie diese kann sehr viel Zeit in Anspruch nehmen!

### Auswahl der Fensterlänge für die gleitenden Durchschnitte

#### WARNUNG: Overfitting

Die Wahl der Fensterlängen wird das Handelssignal, das Sie von Ihrer Crossover-Strategie der gleitenden Durchschnitte erhalten, stark beeinflussen. Es kann bessere Fensterlängen geben, und diese können mit robusten Optimierungstechniken gefunden werden. Augenscheinlich ist es jedoch sehr einfach, die Längen der gleitenden Zeitfenster zu overfitten.